# The plan is to create and analyse the places using zomato api and forsquare and understand the covid data near that place in Bengaluru, a city in India

# Introduction/Business Problem

### The situation of the current pandemic is becoming worse every day. Growing number of cases along with added ignorance by individuals is affecting the recovery rate. So it has become important to understand the number of people around a place is either affected by COVID-19 or is suspected and thus quarantined. This helps people who want to go to a public place or a restaurant. Having data on how many are infected may help analyze the condition of the place. This project aims at one particular location for this - Bengaluru - a city in the southern part of India, in the state of Karnataka.

# Data

### The data used are: 
### 1.  Foursquare location data (to get location data) 
### 2.  Zomato API (to get details of restaurants nearby) 
### 3.  Current COVID -19 dataset from the Karnataka Government website (to analyze current COVID-19 situation)

## Since the kernal was dying, I have kept the limit to 20. It can be increased as required

In [32]:
!pip install folium

In [33]:
import pandas as pd
import folium


Find latitude and longitude of bangalore using geopy

In [34]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Bengaluru"
country ="India"
loc = geolocator.geocode(city+','+ country)
b_lat = loc.latitude
b_long = loc.longitude
print("latitude is :" ,b_lat,"\nlongtitude is:" ,b_long)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is : 12.9791198 
longtitude is: 77.5912997


Checkout bangalore using folium

In [35]:
blore_map = folium.Map(location = [b_lat, b_long], zoom_start = 10)
folium.Marker(
    location=[b_lat, b_long],
    popup='Welcome to <b>BENGALURU</b>',
    tooltip = "Click for more"
).add_to(blore_map)


blore_map

Set credentials for Foursquare

In [41]:
FOURSQUARE_CLIENT_ID = 'U4A4MH53EZ455XB44QAQCVNUB42F05W2H5TAT401KU5R2VKB'
FOURSQUARE_CLIENT_SECRET = '2FMQYPVEAPH5ZPQD1INHF34KL0NRZGJ5EJM54N1NK1AWGMV2'
RADIUS = 5000
NO_OF_VENUES = 20
VERSION = '20200704'

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize
import requests

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(FOURSQUARE_CLIENT_ID, 
                                                                        FOURSQUARE_CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        b_lat, 
                                                                        b_long, 
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    #print(result)
    
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    
    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

    columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))


Total 20 venues fetched within a total radius of 4.0 Km

Total 20 venues fetched


Get Venues from Zomato API

In [44]:
headers = {'user-key': 'b5a7f838a29ed880f18957b65b6ea13f'}
venues_information = []

for index, row in foursquare_venues.iterrows():
    print("Fetching data for venue: {}".format(index + 1))
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    #print(result)
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
#print(venues_information)   
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

Fetching data for venue: 1
Fetching data for venue: 2
Fetching data for venue: 3
Fetching data for venue: 4
Fetching data for venue: 5
Fetching data for venue: 6
Fetching data for venue: 7
Fetching data for venue: 8
Fetching data for venue: 9
Fetching data for venue: 10
Fetching data for venue: 11
Fetching data for venue: 12
Fetching data for venue: 13
Fetching data for venue: 14
Fetching data for venue: 15
Fetching data for venue: 16
Fetching data for venue: 17
Fetching data for venue: 18
Fetching data for venue: 19
Fetching data for venue: 20


In [45]:
b_map = folium.Map(location = [b_lat, b_long], zoom_start = 13)

for name, latitude, longitude in zip(foursquare_venues['name'], foursquare_venues['lat'], foursquare_venues['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(b_map)  

#chandigarh_map.save("maps/Venues by Foursquare.html")
b_map

In [46]:
b_map = folium.Map(location = [b_lat, b_long], zoom_start = 13)

for venue, address, latitude, longitude in zip(zomato_venues['venue'], zomato_venues['address'], 
                                               zomato_venues['latitude'], zomato_venues['longitude']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#cc3535',
        fill_opacity = 0.7,
        parse_html = False).add_to(b_map)  

#b_map.save("maps/Venues by Zomato.html")
b_map

#### Get the newest COVID-19 data of Bengaluru from the government website

In [47]:
b_covid = pd.read_excel("https://karunadu.karnataka.gov.in/hfw/kannada/homequarantivedocs/Bengaluru.xls",index_col = 0)

Just some data cleaning!

In [48]:
b_covid['Street/ Village'] = b_covid['Street/ Village'].apply(str)
b_covid['Tehsil'] = b_covid['Tehsil'].apply(str)
b_covid['District/ City'] = b_covid['District/ City'].apply(str)
b_covid['State'] = b_covid['State'].apply(str)
b_covid['PIN'] = b_covid['PIN'].apply(str)
b_covid['Address'] = b_covid[['Street/ Village', 'Tehsil', 'District/ City','State', 'PIN']].apply(lambda x: ' '.join(x), axis=1)
b_covid.head()

,Date of Arrival,Date until Quarantined at home,Port of Origin of journey,Port of Final Destination,House No,Street/ Village,Tehsil,District/ City,State,PIN,Final District/ City,Address
S.No.,,,,,,,,,,,,
1,2020-03-23,2020-04-06,Dubai,Bangalore,22,Koramangala,nan,Bangalore,Karnataka,560034,"Bengaluru (Urban, Rural & BBMP)",Koramangala nan Bangalore Karnataka 560034
2,2020-03-22,2020-04-05,Dubai,Bangalore,201 A,"22nd A cross, 17 A cross,","3rd sector, HSR layout",Bangalore,Karnataka,560102,"Bengaluru (Urban, Rural & BBMP)","22nd A cross, 17 A cross, 3rd sector, HSR lay..."
3,2020-03-22,2020-04-05,Los Angeles,Bangalore,Parinitha,"Jairam estate, Jakkur",nan,Bangalore,Karnataka,560064,"Bengaluru (Urban, Rural & BBMP)","Jairam estate, Jakkur nan Bangalore Karnataka ..."
4,2020-03-22,2020-04-05,Dubai,Bangalore,2038,Indiranagar,HAL 2nd stage,Bangalore,Karnataka,560008,"Bengaluru (Urban, Rural & BBMP)",Indiranagar HAL 2nd stage Bangalore Karnataka ...
5,2020-03-22,2020-04-05,Seattle,Bangalore,71,GP Rajarampur 9th main,Hanumanthnagar,Bangalore,Karnataka,560019,"Bengaluru (Urban, Rural & BBMP)",GP Rajarampur 9th main Hanumanthnagar Bangalor...


#### Use pgeocode to get latitude and longtitude values for all Pincodes

In [50]:
import pgeocode
import json

nomi = pgeocode.Nominatim('in')
lat = []
long = []
for row in b_covid['PIN']:
    try:
        val = nomi.query_postal_code(row)
        val = val.to_json()
        val = json.loads(val)
        lat.append(val['latitude'])
        long.append(val['longitude'])
    except Exception as e:
        print(e)
        continue
        
print("done")

done


In [51]:
b_covid['Latitude'] = lat
b_covid['Longitude'] = long
b_covid.head()

,Date of Arrival,Date until Quarantined at home,Port of Origin of journey,Port of Final Destination,House No,Street/ Village,Tehsil,District/ City,State,PIN,Final District/ City,Address,Latitude,Longitude
S.No.,,,,,,,,,,,,,,
1,2020-03-23,2020-04-06,Dubai,Bangalore,22,Koramangala,nan,Bangalore,Karnataka,560034,"Bengaluru (Urban, Rural & BBMP)",Koramangala nan Bangalore Karnataka 560034,13.2928,77.641500
2,2020-03-22,2020-04-05,Dubai,Bangalore,201 A,"22nd A cross, 17 A cross,","3rd sector, HSR layout",Bangalore,Karnataka,560102,"Bengaluru (Urban, Rural & BBMP)","22nd A cross, 17 A cross, 3rd sector, HSR lay...",13.0907,77.642300
3,2020-03-22,2020-04-05,Los Angeles,Bangalore,Parinitha,"Jairam estate, Jakkur",nan,Bangalore,Karnataka,560064,"Bengaluru (Urban, Rural & BBMP)","Jairam estate, Jakkur nan Bangalore Karnataka ...",13.1002,77.679686
4,2020-03-22,2020-04-05,Dubai,Bangalore,2038,Indiranagar,HAL 2nd stage,Bangalore,Karnataka,560008,"Bengaluru (Urban, Rural & BBMP)",Indiranagar HAL 2nd stage Bangalore Karnataka ...,12.9910,77.584300
5,2020-03-22,2020-04-05,Seattle,Bangalore,71,GP Rajarampur 9th main,Hanumanthnagar,Bangalore,Karnataka,560019,"Bengaluru (Urban, Rural & BBMP)",GP Rajarampur 9th main Hanumanthnagar Bangalor...,NaN,NaN


In [52]:
foursquare_venues['lat'] = foursquare_venues['lat'].apply(lambda lat: round(float(lat), 4))
foursquare_venues['lng'] = foursquare_venues['lng'].apply(lambda lng: round(float(lng), 4))
zomato_venues['latitude'] = zomato_venues['latitude'].apply(lambda lat: round(float(lat), 4))
zomato_venues['longitude'] = zomato_venues['longitude'].apply(lambda lng: round(float(lng), 4))
b_covid['latitude'] = b_covid['Latitude'].apply(lambda lat: round(float(lat), 4))
b_covid['longitude'] = b_covid['Longitude'].apply(lambda lng: round(float(lng), 4))
b_covid_compressed = b_covid[['latitude','longitude','Address','PIN']]
b_covid_compressed.head()

,latitude,longitude,Address,PIN
S.No.,,,,
1,13.2928,77.6415,Koramangala nan Bangalore Karnataka 560034,560034
2,13.0907,77.6423,"22nd A cross, 17 A cross, 3rd sector, HSR lay...",560102
3,13.1002,77.6797,"Jairam estate, Jakkur nan Bangalore Karnataka ...",560064
4,12.9910,77.5843,Indiranagar HAL 2nd stage Bangalore Karnataka ...,560008
5,NaN,NaN,GP Rajarampur 9th main Hanumanthnagar Bangalor...,560019


In [53]:
dataset = pd.concat([foursquare_venues, zomato_venues], axis = 1)
dataset = dataset.drop(columns = ['name','price_for_two', 'rating','price_range'])
dataset.head()

,categories,lat,lng,venue,latitude,longitude,address
0,Park,12.9770,77.5953,Mathsya Darshini,12.9753,77.5889,"KFDC Ltd, Cubbon Park, K.R.Circle, Lavelle Roa..."
1,Hotel,12.9724,77.5951,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle..."
2,Tea Room,12.9872,77.5948,Infinitea Tea Room & Tea Store,12.9871,77.5948,"2, Shah Sultan Complex, Cunningham Road, Banga..."
3,Hotel,12.9846,77.5849,Masala Klub - The Taj West End,12.9841,77.5840,"Taj West End, Race Course Road, Bangalore"
4,Cricket Ground,12.9781,77.5992,McDonald's,12.9762,77.5984,"10, M/S India Garage, Kasturba Road, MG Road, ..."


In [54]:
dataset['lat_diff'] = dataset['latitude'] - dataset['lat']
dataset['lng_diff'] = dataset['longitude'] - dataset['lng']
selected_venues = dataset[(abs(dataset['lat_diff']) <= 0.0002) & (abs(dataset['lng_diff']) <= 0.0002)].reset_index(drop = True)
selected_venues = selected_venues.drop(columns = ['lat','lng', 'lat_diff','lng_diff'])
selected_venues.head()

,categories,venue,latitude,longitude,address
0,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle..."
1,Tea Room,Infinitea Tea Room & Tea Store,12.9871,77.5948,"2, Shah Sultan Complex, Cunningham Road, Banga..."
2,Shopping Mall,Shiro,12.9718,77.5959,"2nd Floor, UB City Mall, Vittal Mallya Road, L..."
3,Deli / Bodega,Smoke House Deli,12.9717,77.5983,"52/ 53, Ground Floor, Lavelle Road, Bangalore"
4,Italian Restaurant,Toscano,12.9719,77.5964,"2nd Floor, UB City, Vittal Mallya Road, Lavell..."


#### Using geopy get PIN codes

In [55]:
import geopy

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']


geolocator = geopy.Nominatim(user_agent='my-application')

zipcodes = selected_venues.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='longitude')

selected_venues['PIN'] = zipcodes
selected_venues['PIN'] = selected_venues['PIN'].astype(str).str[:6]
selected_venues.head()

,categories,venue,latitude,longitude,address,PIN
0,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001
1,Tea Room,Infinitea Tea Room & Tea Store,12.9871,77.5948,"2, Shah Sultan Complex, Cunningham Road, Banga...",560052
2,Shopping Mall,Shiro,12.9718,77.5959,"2nd Floor, UB City Mall, Vittal Mallya Road, L...",560001
3,Deli / Bodega,Smoke House Deli,12.9717,77.5983,"52/ 53, Ground Floor, Lavelle Road, Bangalore",560001
4,Italian Restaurant,Toscano,12.9719,77.5964,"2nd Floor, UB City, Vittal Mallya Road, Lavell...",560001


In [77]:
merged_Frame = pd.merge(selected_venues,b_covid_compressed, on ='PIN')

In [78]:
merged_Frame

,categories,venue,latitude_x,longitude_x,address,PIN,latitude_y,longitude_y,Address
0,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Banashankri nan Bangalore Karnataka 560001
1,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,H.K.P Road nan Bangalore Karnataka 560001
2,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Race Course Road nan Bengaluru Karnataka 560001
3,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,"High Point 2, Palace Raod nan Bengaluru Karnat..."
4,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,"#402,4th floor Ravi-kirapartmeniplain Street ..."
5,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
6,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
7,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
8,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Mackan Road Bhartinagar Bangalore Karnataka 56...
9,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Lavelle Road nan Bangalore Karnataka 560001


In [79]:
merged_Frame.groupby(['address'])

In [80]:
merged_Frame

,categories,venue,latitude_x,longitude_x,address,PIN,latitude_y,longitude_y,Address
0,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Banashankri nan Bangalore Karnataka 560001
1,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,H.K.P Road nan Bangalore Karnataka 560001
2,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Race Course Road nan Bengaluru Karnataka 560001
3,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,"High Point 2, Palace Raod nan Bengaluru Karnat..."
4,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,"#402,4th floor Ravi-kirapartmeniplain Street ..."
5,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
6,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
7,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Museum Road MG Road Bengaluru Karnataka 560001
8,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Mackan Road Bhartinagar Bangalore Karnataka 56...
9,Hotel,Merak-JW Marriott Hotel,12.9724,77.5951,"JW Marriott, 24/1, Vittal Mallya Road, Lavelle...",560001,12.9914,77.5922,Lavelle Road nan Bangalore Karnataka 560001


#### Group by to see the count of infected individuals around that place

In [91]:
df = merged_Frame.groupby(["address","PIN"]).count()

In [92]:
df

,,categories,venue,latitude_x,longitude_x,latitude_y,longitude_y,Address
address,PIN,,,,,,,
"2, Shah Sultan Complex, Cunningham Road, Bangalore",560052,74,74,74,74,0,0,74
"20/2, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176,176,176,176,176,176,176
"2nd Floor, UB City Mall, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352,352,352,352,352,352,352
"2nd Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352,352,352,352,352,352,352
"52/ 53, Ground Floor, Lavelle Road, Bangalore",560001,176,176,176,176,176,176,176
"JW Marriott, 24/1, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176,176,176,176,176,176,176
"Uber Level, 16th Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352,352,352,352,352,352,352


In [93]:
df = df.drop(columns = ['venue', 'latitude_x','longitude_x','latitude_y','latitude_y','longitude_y','Address'])
df

,,categories
address,PIN,
"2, Shah Sultan Complex, Cunningham Road, Bangalore",560052,74
"20/2, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176
"2nd Floor, UB City Mall, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352
"2nd Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352
"52/ 53, Ground Floor, Lavelle Road, Bangalore",560001,176
"JW Marriott, 24/1, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176
"Uber Level, 16th Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352


In [94]:
df.rename(columns={'categories':'Count of individuals under quarantine'}, inplace=True)
df

,,Count of individuals under quarantine
address,PIN,
"2, Shah Sultan Complex, Cunningham Road, Bangalore",560052,74
"20/2, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176
"2nd Floor, UB City Mall, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352
"2nd Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352
"52/ 53, Ground Floor, Lavelle Road, Bangalore",560001,176
"JW Marriott, 24/1, Vittal Mallya Road, Lavelle Road, Bangalore",560001,176
"Uber Level, 16th Floor, UB City, Vittal Mallya Road, Lavelle Road, Bangalore",560001,352
